### Import libraries

In [1]:
import numpy as np
import pandas as pd

### Get DataFrame from URL

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
data = pd.read_html(url, header = 0)
data

[    Postcode           Borough  \
 0        M1A      Not assigned   
 1        M2A      Not assigned   
 2        M3A        North York   
 3        M4A        North York   
 4        M5A  Downtown Toronto   
 5        M6A        North York   
 6        M6A        North York   
 7        M7A  Downtown Toronto   
 8        M8A      Not assigned   
 9        M9A      Queen's Park   
 10       M1B       Scarborough   
 11       M1B       Scarborough   
 12       M2B      Not assigned   
 13       M3B        North York   
 14       M4B         East York   
 15       M4B         East York   
 16       M5B  Downtown Toronto   
 17       M5B  Downtown Toronto   
 18       M6B        North York   
 19       M7B      Not assigned   
 20       M8B      Not assigned   
 21       M9B         Etobicoke   
 22       M9B         Etobicoke   
 23       M9B         Etobicoke   
 24       M9B         Etobicoke   
 25       M9B         Etobicoke   
 26       M1C       Scarborough   
 27       M1C       

### Get data based on headings

In [31]:
headings = ['Postcode','Borough','Neighborhood']
for tables in data:
    current_headings = tables.columns.values[:4]
    if all(current_headings == headings):
        break

In [32]:
tables.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
table['Borough'].unique()

array(['Not assigned', 'North York', 'Downtown Toronto', "Queen's Park",
       'Scarborough', 'East York', 'Etobicoke', 'York', 'East Toronto',
       'West Toronto', 'Central Toronto', 'Mississauga'], dtype=object)

In [7]:
### Drop 'Not assigned' in Borough columns

In [8]:
process_table = table[table['Borough']!='Not assigned'].reset_index().drop('index', axis=1)
process_table.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [9]:
process_table['Borough'].unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Scarborough',
       'East York', 'Etobicoke', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

### Join the Neighborhood based on similar Postcode with seperated units = ','

In [10]:
df = pd.DataFrame(process_table.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join)).reset_index()
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Change the 'Not assigned' on 'Neighborhood' columns = values in 'Borough' colunmns

In [11]:
df.loc[df['Neighborhood']=='Not assigned','Neighborhood'] = df.loc[df['Neighborhood']=='Not assigned','Borough']

In [13]:
df.shape

(103, 3)

### Use beautiful soup

In [15]:
from bs4 import BeautifulSoup
import requests

In [19]:
page_response = requests.get(url)
page_content = BeautifulSoup(page_response.content,'html.parser')

In [43]:
table = page_content.find('table',{'class':'wikitable sortable'})
rows = table.find_all('tr')
raw_df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])

In [49]:
for row in rows:
    row_items = row.find_all('td')
    if len(row_items) > 0:
        postcode = row_items[0].text.strip()
        borough = row_items[1].text.strip()
        if borough.lower() != 'not assigned':
            neighborhood = row_items[2].text.strip()
            raw_df = raw_df.append({'PostalCode':postcode, 
                                    'Borough':borough, 
                                    'Neighborhood':neighborhood}, 
                                   ignore_index = True)
raw_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
